In [9]:
from gensim import corpora, models, similarities
import scipy

In [3]:
dictionary = corpora.Dictionary.load('datadump/lsi_data/train.dict')
corpus = corpora.MmCorpus('datadump/lsi_data/train_corpora.mm')

In [4]:
print(corpus)

MmCorpus(354748 documents, 20143 features, 1450041 non-zero entries)


In [5]:
# lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=300)

In [32]:
# lsi.save("datadump/lsi_data/lsi_model")  

In [ ]:
loaded_model = models.LsiModel.load("datadump/lsi_data/lsi_model")

In [22]:
def get_lsi_vector(sentence,dict_lsi,lsi_model):
    vec_bow1 = dict_lsi.doc2bow(sentence.lower().split())
    vec_lsi1 = lsi_model[vec_bow1]
    return  [ k[1] for k in vec_lsi1]

In [79]:
def get_lsi_cosine(s1,s2):
    dist_comp = scipy.spatial.distance.cosine(get_lsi_vector(s1,dictionary,lsi),get_lsi_vector(s2,dictionary,lsi))
    if dist_comp <=1.0 and dist_comp>=0.0:
        return dist_comp
    return 1.0

In [68]:
doc1 = "how do i become a good data scientist"
 # convert the query to LSI space
doc2 = "how to become good machine learning engineer"

In [69]:
# scipy.spatial.distance.cosine(get_lsi_vector(doc1,dictionary,lsi),get_lsi_vector(doc2,dictionary,lsi))

In [70]:
import csv

In [80]:
sim_score_true = []
sim_score_false = []
with open('datadump/lsi_data/train.csv' ,"r") as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        q1 = row["question1"]
        q2 = row["question2"]
        is_dup = row["is_duplicate"]
        try:
            sim_scr = get_lsi_cosine(q1,q2)
            if is_dup == '0':
                sim_score_false.append(sim_scr)
            elif is_dup == '1':
                sim_score_true.append(sim_scr)
        except:
            #print(q1)
            #print(q2)
            continue
        

In [81]:
len(sim_score_true), len(sim_score_false)

(88421, 148585)

In [96]:
def get_ave(ls):
    return sum(ls)*1.0/len(ls)

In [97]:
thresh_ = get_ave( [ get_ave(sim_score_true),get_ave(sim_score_false)] )

In [98]:
thresh_

0.4797853006671452

In [99]:
res_ = []
with open('datadump/lsi_data/test.csv' ,"r") as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        q1 = row["question1"]
        q2 = row["question2"]
        is_dup =  row["is_duplicate"]=='1' 
        is_dup_pred = False
        try:
            sim_scr = get_lsi_cosine(q1,q2)
            is_dup_pred = sim_scr<thresh_
        except:
            is_dup_pred = False
        if is_dup == is_dup_pred:
            res_.append(1)
        else:
            res_.append(0)

In [113]:
print("accuracy :",sum(res_)/len(res_))

accuracy : 0.6466973886328725


In [108]:
is_dup_l = []
with open('datadump/lsi_data/test.csv' ,"r") as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        is_dup =  0.0
        if row["is_duplicate"]=='1':
            is_dup =  1.0
        is_dup_l.append(is_dup)

In [112]:
1-get_ave(is_dup_l)

0.6284376393637581